# Before we start...

In [1]:
import urllib

imdb_url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
urllib.urlretrieve(imdb_url, './aclImdb_v1.tar.gz') 

('./aclImdb_v1.tar.gz', <httplib.HTTPMessage instance at 0x110e56ea8>)

In [2]:
glove_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
urllib.urlretrieve(glove_url, './glove.6B.zip')

('./glove.6B.zip', <httplib.HTTPMessage instance at 0x110f1c050>)

In [3]:
imdb_dir = './aclImdb'
glove_dir = './glove.6B'

In [4]:
import tarfile
tar = tarfile.open('./aclImdb_v1.tar.gz', "r:gz")
tar.extractall()
tar.close()

In [6]:
import zipfile
import os
if not os.path.exists(glove_dir):
    os.mkdir(glove_dir)
zip_ref = zipfile.ZipFile('./glove.6B.zip', 'r')
zip_ref.extractall(glove_dir)
zip_ref.close()

# Text-Preprocessing

In [7]:
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [8]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.2'

In [9]:
n_data = len(texts)

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = int(n_data* 0.8) 
val_samples = n_data - training_samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

In [11]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 88584 unique tokens.


In [13]:
data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: ]
y_val = labels[training_samples: ]

('Shape of data tensor:', (25000, 100))
('Shape of label tensor:', (25000,))


In [14]:
x_train.shape[0], y_train.shape[0]

(20000, 20000)

In [15]:
x_val.shape[0], y_val.shape[0]

(5000, 5000)

In [16]:
test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [17]:
x_test.shape[0], y_test.shape[0]

(25000, 25000)

# Text-Vectorization

## Word Embedding

### Using pre-trained embedding (Glove)

In [18]:
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [19]:
embedding_dim = 50

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [20]:
if_using_pretrained = True

# CNN 

In [21]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Convolution1D, Dropout


model_cnn = Sequential()
model_cnn.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model_cnn.add(Convolution1D(2, 2))
model_cnn.add(Convolution1D(2, 3))
model_cnn.add(Flatten())
model_cnn.add(Dense(32, activation='relu'))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 99, 2)             202       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 97, 2)             14        
_________________________________________________________________
flatten_1 (Flatten)          (None, 194)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                6240      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total para

In [22]:
if if_using_pretrained:
    model_cnn.layers[0].set_weights([embedding_matrix])
    model_cnn.layers[0].trainable = False

In [23]:
model_cnn.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model_cnn.fit(x_train, y_train,
                    epochs=5,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 4s 205us/step - loss: 0.6610 - acc: 0.5894 - val_loss: 0.5702 - val_acc: 0.7096

In [24]:
model_cnn.evaluate(x_test, y_test)

25000/25000 [==============================] - 2s 69us/step

[0.55918194041728975, 0.71140000000000003]

# RNN

In [25]:
from keras.layers import SimpleRNN

In [26]:
model_rnn = Sequential()
model_rnn.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model_rnn.add(SimpleRNN(32, return_sequences=True))
model_rnn.add(SimpleRNN(32))  # This last layer only returns the last outputs.
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100, 32)           2656      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 504,769
Trainable params: 504,769
Non-trainable params: 0
_________________________________________________________________


In [27]:
if if_using_pretrained:
    model_rnn.layers[0].set_weights([embedding_matrix])
    model_rnn.layers[0].trainable = False

In [28]:
model_rnn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model_rnn.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_data=(x_val, y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 9s 436us/step - loss: 0.6857 - acc: 0.5621 - val_loss: 0.6663 - val_acc: 0.6044

In [29]:
model_rnn.evaluate(x_test, y_test)

25000/25000 [==============================] - 9s 360us/step

[0.56723626913070679, 0.71372000000000002]

# LSTM

In [30]:
from keras.layers import LSTM

model_lstm = Sequential()
model_lstm.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model_lstm.add(LSTM(32))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


In [31]:
if if_using_pretrained:
    model_lstm.layers[0].set_weights([embedding_matrix])
    model_lstm.layers[0].trainable = False

In [32]:
model_lstm.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model_lstm.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_data=(x_val, y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 16s 807us/step - loss: 0.6355 - acc: 0.6317 - val_loss: 0.5769 - val_acc: 0.7112

In [33]:
model_lstm.evaluate(x_test, y_test)

25000/25000 [==============================] - 10s 404us/step

[0.50366033738613125, 0.75404000000000004]